*   **Dataset**: CHB
*   **Processing**: corelation coefficient

*   **Epochs**: 30  preictal+ictal
*   **Kernels**: WL, Spectral Decomposition, Random Walk
*   **Classifiers**: Decision Tree, Random Forest, SVC




In [1]:
!pip install mne
!pip install graphkit-learn

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import mne

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from os import listdir
from os.path import isfile, join
import tqdm
import multiprocessing
import networkx as nx
from collections import Counter, defaultdict
#from grakel.kernels import RandomWalk
from gklearn.utils.kernels import deltakernel, gaussiankernel, kernelproduct
import functools
from multiprocessing.pool import ThreadPool as Pool

from sklearn.decomposition import PCA,KernelPCA

import scipy.signal as sig
from scipy import linalg, stats
import seaborn as sns


import datetime
timestamp = datetime.datetime.now()
timestamp = timestamp.timestamp() * 1000
timestamp = str(timestamp)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 14.5 MB/s eta 0:00:00


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("drive/My Drive/Journal_Executions")

Mounted at /content/drive


In [29]:
#Repeat for CHB data
# fpath=['/content/gdrive/MyDrive/ProjectData/chb02']
# fpath="C:\\Users\\Owner\\OneDrive - University of New Brunswick\\Thesis\\Code\\graphKKE\\mPLV\\Data_EDF\\chb02"

ictalfp=[];ictalf=[];seizest=[];seizen=[];
preictalfp=[];preictalf=[];preseizest=[];preseizen=[];
interictalfp=[];interictalf=[];interseizest=[];interseizen=[];

SPH = [0,20]
SPHl = SPH[0]*60;
SPHu = SPH[1]*60;
ch=[0, 22];
MINSPHu=SPH[0];
MAXSPHl=SPH[1];

fpath = ['Data_EDF/chb02']

for f in fpath:
    print('f', f)
    print('f', f.split('/'))
    p=f.split('/')[-1]
    print('p', p)
    print('f', f)
    f1=f+'/'+p+'-summary.txt'
    print('p', f1)
    f2 = open(f1,'r+')
    string_list=f2.readlines()
    print(len(string_list))
    i=0;#flag=0
    fs = string_list[0].split()[-2]
    while (string_list[i][0]!='F'):
        i=i+1
    while (i<len(string_list)): #(flag!=1):
        if (string_list[i+3][len(string_list[i+3])-2]=='0'):
              i=i+5
        else:
            d=string_list[i].split(); ictal_fname= d[2]
            d=string_list[i+4].split(); st_t = int(d[3])
            d=string_list[i+5].split(); en_t = int(d[3])

          #CREATE ICTAL LIST
            ictalfp.append(f+'/'+ictal_fname);
            ictalf.append(ictal_fname);
            seizest.append(st_t);seizen.append(en_t)

          #CREATE PREICTAL as SPHu min = SPHu*60 sec before and upto the seizure start point if available,
          # else include from previous file

            if (st_t-SPHu>=0):
                pst=st_t-SPHu; pen=st_t-SPHl;
                preictalfp.append(f+'/'+ictal_fname);preictalf.append(ictal_fname);preseizest.append(pst);preseizen.append(pen)

            #ADD REMAINING FRONT DATA TO INTERICTAL
                if (st_t-SPHu >0):
                    intst=0; inten=st_t-SPHu; #DATA BEFORE PREICTAL
                    interictalfp.append(f+'/'+ictal_fname);interictalf.append(ictal_fname);interseizest.append(intst);interseizen.append(inten)
                #ADD REMAINING TAIL DATA TO INTERICTAL IF NEXT FILE IS NON-SEIZURE FILE
                if (string_list[i+10][len(string_list[i+3])-2]=='0'):
                    intst=int(en_t); inten=3600;#DATA AFTER SEIZURE END UPTO END of FILE
                    interictalfp.append(f+'/'+ictal_fname);interictalf.append(ictal_fname);interseizest.append(intst);interseizen.append(inten)

            else:
                #IF SUFFICIENT (30 min) PREICTAL DATA NOT AVAILABLE IN THE CURRENT FILE THEN PICK FROM CURRENT FILE and FROM PREVIOUS FILE IF AVAILABLE
                pst=0;  pen=st_t-SPHl;
                preictalfp.append(f+'/'+ictal_fname);preictalf.append(ictal_fname);preseizest.append(pst);preseizen.append(pen)
                remainlength =SPHu-st_t
                #IF PREVIOUS FILE IS NON-SEIZURE FILE PICK REMAINING secs without any further CHECK

                if (string_list[i-2][len(string_list[i+3])-2]=='0'):
                    d=string_list[i-5].split(); preictal_fname= d[2] #NAME OF PREVIOUS FILE
                    pst=3600-remainlength;  pen=3600; #Assuming each file MITPHYSIONET having 60 min recording- to be customized for other set-up
                    preictalfp.append(f+'/'+preictal_fname);preictalf.append(preictal_fname);preseizest.append(pst);preseizen.append(pen)

                      #ADD REMAINING FRONT DATA OF PREVIOUS NON SEIZURE FILE TO INTERICTAL
                    intst=0; inten=pst-1;#DATA BEFORE PREICTAL
                    interictalfp.append(f+'/'+preictal_fname);interictalf.append(preictal_fname);interseizest.append(intst);interseizen.append(inten)

                      #ADD REMAINING  DATA OF PREVIOUS ALL NON SEIZURE FILES TO INTERICTAL
                    j=i-7 #Last line of file preceding previous file
                    while(string_list[j][len(string_list[j])-2]=='0'): #will carry 0 only if it is Non-Seizure file
                        d=string_list[j-3].split(); interictal_fname= d[2] #NAME OF NON-SEIZURE FILE
                        intst=0; inten=3600;#ENTIRE DATA AS INTERICTAL
                        interictalfp.append(f+'/'+interictal_fname);interictalf.append(interictal_fname);interseizest.append(intst);interseizen.append(inten)
                        j=j-5

                else:
                  #OTHERWISE PICK FROM PREVIOUS (SEIZURE) FILE IF SUFFICIENT LENGTH AFTER SEIZURE END AVAILABLE-
                    d=string_list[i-7].split(); preictal_fname= d[2] #NAME OF PREVIOUS FILE
                    d=string_list[i-2].split(); enprev = int(d[3]) # #get seizure end time of previous file
                    available=3600-enprev
                    if (available >= remainlength):
                        pst=3600-remainlength;  pen=3600;
                        preictalfp.append(f+'/'+preictal_fname);preictalf.append(preictal_fname);preseizest.append(pst);preseizen.append(pen)

                        #ADD REMAINING DATA OF PREVIOUS SEIZURE FILE TO INTERICTAL
                        if (pst>enprev+1):
                            intst=enprev+1; inten=pst;#DATA BETWEEN SEIZURE END OF PREVIOUS FILE and THE Start of PREICTAL
                            interictalfp.append(f+'/'+preictal_fname);interictalf.append(preictal_fname);interseizest.append(intst);interseizen.append(inten)

                  #IF SUFFICIENT LENGTH NOT AVAILABLE, then RECORD THE AVAILABLE LENGTH AS MINSPH
                    else:
                        pst=enprev;  pen=3600; MINSPHu= SPH[1]-remainlength+ available
                        preictalfp.append(f+'/'+preictal_fname);preictalf.append(preictal_fname);preseizest.append(pst);preseizen.append(pen)
            i=i+7

    f2.close()


dict={'ictal_fpath':ictalfp, 'ictal_f':ictalf , 'seiz_st':seizest , 'seiz_en':seizen}
df = pd.DataFrame(dict)
#print('ICTAL\n', df)

dict={'Preictal_fpath':preictalfp, 'Preictal_f':preictalf , 'Preseiz_st':preseizest , 'Preseiz_en':preseizen}
dfpre = pd.DataFrame(dict)
#print('Preictal\n',dfpre)

print(len(df),len(dfpre))

#===================================================================================
#Data Selection with  Band- Pass Filter
band1= [0,5]; band2=[5, 8]; band3= [8,13]; band4= [13, 21]; band5= [21, 34]; band6= [34,100]

fs= 256; #st_f=band4[0]; en_f=band4[1];
st_f=0; en_f=100
train_data=[]; train_labels=[]

ictald=np.empty((22,0))
preictald=np.empty((22,0))
interictald=np.empty((22,0))

#NOW USE ABOVE DATAFRAMES AND READ FILES TO FORM EPOCHS and TRAINING SET
#ICTAL
for i in range(0,len(df)):
    f=df['ictal_fpath'][i]
    raw = mne.io.read_raw_edf(f, verbose=False)
    raw.load_data() ## Filtering requires the data to be loaded to the memory
    d=raw.filter(None, 50., fir_design='firwin')#power noise notch filter at 50Hz
    d=raw.copy().filter(l_freq=st_f, h_freq=en_f)
    st=df['seiz_st'][i]; en=df['seiz_en'][i];
    ictald=np.concatenate((ictald,d._data[0:22, st*fs:en*fs]), axis=1)
#PREICTAL
for i in range(0,len(dfpre)):
    f=dfpre['Preictal_fpath'][i]
    raw = mne.io.read_raw_edf(f, verbose=False)
    raw.load_data() ## Filtering requires the data to be loaded to the memory
    d=raw.filter(None, 50., fir_design='firwin') #power noise notch filter at 50Hz
    d=raw.copy().filter(l_freq=st_f, h_freq=en_f)
    st=dfpre['Preseiz_st'][i]; en=dfpre['Preseiz_en'][i];
    preictald=np.concatenate((preictald,d._data[0:22, st*fs:en*fs]), axis=1)


#Transform using z-score
from scipy import stats
ictald=stats.zscore(ictald, axis=1)
preictald=stats.zscore(preictald, axis=1)

print('Ictal: ', ictald.shape, '\nPreictal: ', preictald.shape)

#Create Train Data - 2s epochs from each category with corresponding label {1-preictal, 0- interictal}
fs=256
epochsize=2*fs

#Similarly create epochs for preictal
epochno =preictald.shape[1]//epochsize
preictalepochs = np.array(np.hsplit(preictald[:,0:epochno*epochsize],epochno ))
print('Preictal epochs:',epochno, '\n', preictalepochs.shape)

#Similarly create epochs for Ictal
epochno =ictald.shape[1]//epochsize
ictalepochs = np.array(np.hsplit(ictald[:,0:epochno*epochsize],epochno ))
print('Ictal epochs:',epochno, '\n', ictalepochs.shape)



f Data_EDF/chb02
f ['Data_EDF', 'chb02']
p chb02
f Data_EDF/chb02
p Data_EDF/chb02/chb02-summary.txt
215
3 4
Reading 0 ... 245503  =      0.000 ...   958.996 secs...


<ipython-input-29-022773aa6cfa>:135: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(f, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 69 samples (0.270 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 35 samples (0.137 s)



...

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 69 samples (0.270 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.5s


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 35 samples (0.137 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.6s


Ictal:  (22, 44032) 
Preictal:  (22, 921600)
Preictal epochs: 1800 
 (1800, 22, 512)
Ictal epochs: 86 
 (86, 22, 512)


In [59]:
fs=256
num_channels=preictalepochs.shape[1]

nanct=0
NEpoch=30

adj_matrix = np.zeros((NEpoch*2, num_channels, num_channels))
labels=np.zeros(NEpoch*2)

k=0
for i in np.arange(0,NEpoch):
    epoch = preictalepochs[i,:,:] #channel_data[:,3*i*fs:3*(i+1)*fs]
    cor_mat = np.corrcoef(epoch) #replace with mPLV compPlv(epoch) #
    if(np.isnan(cor_mat).any()):
        nanct=nanct+1
        continue
    adj_matrix[k] = cor_mat
    labels[k]=0
    k=k+1

nanct=0
NEpoch=30

for i in np.arange(0,NEpoch):
    epoch = ictalepochs[i,:,:] #channel_data[:,3*i*fs:3*(i+1)*fs]
    cor_mat = np.corrcoef(epoch) #replace with mPLV compPlv(epoch) #
    if(np.isnan(cor_mat).any()):
        nanct=nanct+1
        continue
    adj_matrix[k] = cor_mat
    labels[k]=1
    k=k+1

#Now Binarize Adjacency matrices before feeding for graph preparation
for i in np.arange(0,k):
    th=np.percentile(adj_matrix[i].flatten(),25)
    adj_matrix[i][adj_matrix[i]<th]=0
    adj_matrix[i][adj_matrix[i]>=th]=1

print(adj_matrix.shape)
adj_matrix= adj_matrix.astype(int)

(60, 22, 22)


In [60]:
# Generate shuffle indices for the first axis

print('labels::::', labels)
print('adj_matrix::::', adj_matrix.shape)
shuffle_indices = np.random.permutation(adj_matrix.shape[0])

print('shuffle_indices::::', shuffle_indices)
# Shuffle the array along the first axis using the generated indices
adj_matrix = adj_matrix[shuffle_indices]
labels = labels[shuffle_indices]

print('labels::::', labels)
print('adj_matrix::::', adj_matrix.shape)

labels:::: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
adj_matrix:::: (60, 22, 22)
shuffle_indices:::: [22 57 55 19 37 14 46 12  6 20 58 30 23 16 42 10 52 56 47 25 43 18 21 54
 41 53 59 48 35  8  7 27 50  4 32 51  1 15 36 40  5 26 11 31 44 49 29 38
 39 45  9 34 17  3 13 33  2 24  0 28]
labels:::: [0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.]
adj_matrix:::: (60, 22, 22)


In [67]:
#adj_matrix = np.random.randint(0, 100, size=(20, 20, 20))#datafile
#adj_matrix computed above
num_nodes = adj_matrix.shape[0]
node_labels = list(range(1, num_nodes+1))
graphs = []

for graph in adj_matrix:
    graph = nx.from_numpy_array(graph)
    mapping = {i: label for i, label in enumerate(node_labels)}
    graph = nx.relabel_nodes(graph, mapping)
    graphs.append(graph)

from gklearn.kernels.weisfeilerLehmanKernel import weisfeilerlehmankernel
kernel_matrix = weisfeilerlehmankernel(
    graphs,
    node_label='atom',
    edge_label='bond_type',
    height=2,
    base_kernel='subtree',
    parallel=None,
    n_jobs=None,
    verbose=True)
kernel = 'WL'

'''
from gklearn.kernels import SpectralDecomposition
graph_kernel = SpectralDecomposition(
    ds_infos=graphs,
    weight=1e-2,
    p=None,
    q=None,
    edge_weight=None,
    sub_kernel='geo'
)
kernel = 'SD'
kernel_matrix = graph_kernel.compute(
    graphs,
    parallel=None,
    n_jobs=multiprocessing.cpu_count(),
    verbose=True
)

from gklearn.kernels.randomWalkKernel import randomwalkkernel
from gklearn.utils.kernels import deltakernel, gaussiankernel, kernelproduct
import functools
import multiprocessing

kernel = 'RW'
mixkernel = functools.partial(kernelproduct, deltakernel, gaussiankernel)
sub_kernels = [{'symb': deltakernel, 'nsymb': gaussiankernel, 'mix': mixkernel}]
sub_kernel = ['geo', 'exp']
compute_method = 'conjugate'

kernel = 'RandomWalk'
kernel_matrix = randomwalkkernel(
			graphs,
			compute_method=compute_method,
			weight=0.001,
			p=None,
			q=None,
			edge_weight=None,
			node_kernels=sub_kernels,
			edge_kernels=sub_kernels,
			node_label='atom',
			edge_label='bond_type',
			sub_kernel=sub_kernel,
			n_jobs=multiprocessing.cpu_count(),
			verbose=True
		)
'''


 --- Weisfeiler-Lehman subtree kernel matrix of size 60 built in 0.09592866897583008 seconds ---


"\nfrom gklearn.kernels import SpectralDecomposition\ngraph_kernel = SpectralDecomposition(\n    ds_infos=graphs,\n    weight=1e-2,\n    p=None,\n    q=None,\n    edge_weight=None,\n    sub_kernel='geo'\n)\nkernel = 'SD'\nkernel_matrix = graph_kernel.compute(\n    graphs,\n    parallel=None,\n    n_jobs=multiprocessing.cpu_count(),\n    verbose=True\n)\n\nfrom gklearn.kernels.randomWalkKernel import randomwalkkernel\nfrom gklearn.utils.kernels import deltakernel, gaussiankernel, kernelproduct\nimport functools\nimport multiprocessing\n\nkernel = 'RW'\nmixkernel = functools.partial(kernelproduct, deltakernel, gaussiankernel)\nsub_kernels = [{'symb': deltakernel, 'nsymb': gaussiankernel, 'mix': mixkernel}]\nsub_kernel = ['geo', 'exp']\ncompute_method = 'conjugate'\n\nkernel = 'RandomWalk'\nkernel_matrix = randomwalkkernel(\n\t\t\tgraphs,\n\t\t\tcompute_method=compute_method,\n\t\t\tweight=0.001,\n\t\t\tp=None,\n\t\t\tq=None,\n\t\t\tedge_weight=None,\n\t\t\tnode_kernels=sub_kernels,\n\t\t

In [68]:
# kernel_matrix[0].min()
# print('X VALUE::::::', X.shape)
# # print('y VALUE::::::::::::', len(y))


#https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/

# from sklearn.preprocessing import normalize
# kernel_matrix_normalized = normalize(kernel_matrix[0], axis=1)

kpca = KernelPCA(kernel='rbf', gamma=1)
gram_kpca = kpca.fit_transform(kernel_matrix[0])

X=gram_kpca
y=labels

print('X VALUE::::::', X.shape)
print('y VALUE::::::::::::', len(y))

print("Feature Variability:", np.var(X, axis=0))
print("Label Distribution:", np.unique(y, return_counts=True))
print("Explained Variance (All Components):", kpca.eigenvalues_)

# import matplotlib.pyplot as plt
# plt.scatter(X[:, 0], np.zeros_like(X[:, 0]), c=y, cmap='viridis')
# plt.colorbar()
# plt.title("Feature Distribution After Kernel PCA")
# # plt.show()
# savefigure(plt, 'feature_distribution', kernel)

X VALUE:::::: (60, 1)
y VALUE:::::::::::: 60
Feature Variability: [0.03277778]
Label Distribution: (array([0., 1.]), array([30, 30]))
Explained Variance (All Components): [1.96666667]


In [69]:
print('Kernel:::', kernel)

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

param_grid_svc = {
  'C': [0.01, 0.1, 1, 10, 100],
  'kernel': ['linear', 'rbf', 'poly'],
  'gamma': ['scale', 'auto'],
  'class_weight': ['balanced', None]
}

param_grid_rf = {
  'n_estimators': [50, 100, 200],
  'max_depth': [None, 10, 20, 30],
  'min_samples_split': [2, 5, 10],
  'max_features': ['sqrt', 'log2'],  # Number of features to consider for splits
  'class_weight': ['balanced', None]
}

param_grid_dt = {
  'max_depth': [None, 10, 20, 30],
  'min_samples_split': [2, 5, 10],
  'class_weight': ['balanced', None]
}

from sklearn.utils.class_weight import compute_class_weight
svc = SVC(class_weight='balanced', random_state=42)
rf = RandomForestClassifier(oob_score=True, class_weight='balanced', random_state=42)
dt = DecisionTreeClassifier(class_weight='balanced', random_state=42)


# Perform grid search for each model
grid_search_svc = GridSearchCV(svc, param_grid_svc, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search
grid_search_svc.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)
grid_search_dt.fit(X_train, y_train)

# Get the best estimators
best_svc = grid_search_svc.best_estimator_
best_rf = grid_search_rf.best_estimator_
best_dt = grid_search_dt.best_estimator_

print('best_svc', best_svc)
print('best_rf', best_rf)
print('best_dt', best_dt)

# Evaluate each model and print accuracy and AUC
models = { 'Decision Tree': best_dt, 'Random Forest': best_rf, 'SVC': best_svc}
results = {}

for name, model in models.items():
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else y_pred

    accuracy = accuracy_score(y_test, y_pred)
    precision= precision_score(y_test, y_pred, average='weighted')
    recall= recall_score(y_test, y_pred, average='weighted')
    f1=f1_score(y_test, y_pred, average='weighted')
    auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')

    results[name] = {'accuracy': accuracy, 'AUC': auc_score, 'precision':precision,'recall': recall,'f1': f1}
    print(f"{name} - Accuracy: {accuracy:.4f}, AUC: {auc_score:.4f}, precision:{precision:.4f},recall: {recall:.4f},f1: {f1:.4f}")

# Plot ROC Curves
# plt.figure(figsize=(10, 8))

# for name, model in models.items():
#     y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else y_pred
#     fpr, tpr, _ = roc_curve(y_test, y_pred_proba, pos_label=1)
#     roc_auc = auc(fpr, tpr)
#     plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.2f})')



Kernel::: WL


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
10 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py", line 275, in fit
    raise ValueError(
ValueError: The dual coefficients or intercepts are not finite. The

best_svc SVC(C=0.01, kernel='linear', random_state=42)
best_rf RandomForestClassifier(class_weight='balanced', n_estimators=50, oob_score=True,
                       random_state=42)
best_dt DecisionTreeClassifier(class_weight='balanced', random_state=42)
Decision Tree - Accuracy: 0.2500, AUC: 0.5000, precision:0.0625,recall: 0.2500,f1: 0.1000
Random Forest - Accuracy: 0.2500, AUC: 0.5000, precision:0.0625,recall: 0.2500,f1: 0.1000
SVC - Accuracy: 0.2500, AUC: 0.5000, precision:0.0625,recall: 0.2500,f1: 0.1000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
